# Time Causal VAE

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import yaml
sys.path.append('../src')
sys.path.append('../../../..')
# sys.path.append('../../../')

from tsvae.utils.logger_utils import get_console_logger
logger = get_console_logger(__name__)
from os import path as pt
from experiments.experiment_utils import get_output_dir, update_config
from tsvae.dataset.data_pipeline import DataPipeline
from tsvae.models.network_pipeline import NetworkPipeline
from tsvae.utils.random_utils import set_seed
from tsvae.trainers.base_trainer_config import BaseTrainerConfig
from tsvae.trainers.training_pipeline import TrainingPipeline
from evaluations.hyperparameter import ModelEvaluator
from os import path as pt
import pandas as pd

from data_processing.post_processing import post_processing
from data_processing.config import post_processing_config
from models.DeepHedger import hedging
import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'post_processing_config' from 'config' (/Users/sondrerogde/Dev/master-thesis/src/models/data_generating_models/TimeCausalVAE/notebooks/../../../../config.py)

## TCVAE Model

In [7]:
base_output_dir = '..' 
new_config = {'epochs' : 100, 'wandb': False, 'comment': 'test'}
exp_config_path = '../configs/Master/betacvae.yaml'

with open(exp_config_path) as file:
    exp_config = dict(yaml.safe_load(file))

exp_config = update_config(exp_config, new_config)

exp_config.base_output_dir = base_output_dir
exp_config.output_dir = get_output_dir(exp_config)
logger.info(f"Experiment results saved to {exp_config.output_dir}")

logger.info(f"Saving experiment config to {exp_config.output_dir}")
config_file_path = pt.join(exp_config.output_dir, "exp_config.yaml")
with open(config_file_path, "w") as outfile:
    yaml.dump(exp_config, outfile, default_flow_style=False)
# logger.info(exp_config)

# Generating data
logger.info(f"Setting ramdom seed: {exp_config.seed}")
set_seed(exp_config.seed)

Experiment results saved to ../results/Master_timestep_30/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Experiment results saved to ../results/Master_timestep_30/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Experiment results saved to ../results/Master_timestep_30/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Experiment results saved to ../results/Master_timestep_30/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Experiment results saved to ../results/Master_timestep_30/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Experiment results saved to ../results/Master_timestep_30/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Saving experiment config to ../results/Master_timestep_30/model_BetaCVAE_De_CLSTMRes_En_CLSTMRes_Prior_RealNVP_Con_Id_Dis_None_comment_test
Saving experiment co

In [8]:
logger.info(f"Loading dataset: {exp_config.dataset}")
data_pipeline = DataPipeline()
train_dataset, eval_dataset = data_pipeline(exp_config)

# Loading network
logger.info("Load networks:")
network_pipeline = NetworkPipeline()
model = network_pipeline(exp_config)
# logger.info(f"{model}")

Loading dataset: Master
Loading dataset: Master
Loading dataset: Master
Loading dataset: Master
Loading dataset: Master
Loading dataset: Master
Base dataset initialized
Load networks:
Load networks:
Load networks:
Load networks:
Load networks:
Load networks:


In [9]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sondre-rogde (sondre-rogde-ntnu). Use `wandb login --relogin` to force relogin


True

In [10]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.0001,
            'max': 0.02
        },
        'E_hidden_dim': {
            'values': [8, 16, 32]
        },
        'E_num_layers': {
            'values': [1, 2, 3]
        },
        'D_hidden_dim': {
            'values': [8, 16, 32]
        },
        'D_num_layers': {
            'values': [1, 2, 3]
        }
    }
}


sweep_id = wandb.sweep(sweep_config, project="tcvae_hyperopt")

Create sweep with ID: tfoy3dgo
Sweep URL: https://wandb.ai/sondre-rogde-ntnu/tcvae_hyperopt/sweeps/tfoy3dgo


In [ ]:
def objective():
    wandb.init()
    config = wandb.config
    
    exp_config.learning_rate = config.learning_rate
    exp_config.E_hidden_dim = config.E_hidden_dim
    exp_config.E_num_layers = config.E_num_layers
    exp_config.D_hidden_dim = config.D_hidden_dim
    exp_config.D_num_layers = config.D_num_layers

    training_config = BaseTrainerConfig(
    output_dir=exp_config.output_dir,
    learning_rate=exp_config.lr,
    per_device_train_batch_size=exp_config.train_batch_size,
    per_device_eval_batch_size=exp_config.eval_batch_size,
    optimizer_cls=exp_config.optimizer,
    optimizer_params=None,
    scheduler_cls=None,
    scheduler_params=None,
    steps_saving=exp_config.steps_saving,
    steps_predict=exp_config.steps_predict,
    seed=exp_config.seed,
    num_epochs=exp_config.epochs,
    wandb_callback=exp_config.wandb,
    wandb_output_dir=exp_config.base_output_dir + "/wandb",
)


    # Train TCVAE
    train_pipeline = TrainingPipeline(model=model, training_config=training_config, exp_config=exp_config)

    trainer = train_pipeline(
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        device_name=exp_config.device_name,
    )

    train_pipeline.train(log_output=True)

    # Generate data

    model_dir = pt.join(trainer.training_dir, 'final_model')
    model_evaluator = ModelEvaluator(model_dir)
    output_dir = model_evaluator.hyper_model_dir
    model = model_evaluator.model

    df_tot_fake = pd.DataFrame()
    # 100 times 1000 corresponds to M = 100.000
    for i in range(100):
        test_data, gen_data, recon_data = model_evaluator.load_data(seed = i, n_sample_test=1000)
        ds = model_evaluator.data_ppl.base_dataset

        real_data = test_data
        recon_data = recon_data
        fake_data = gen_data

        df_sub_fake = pd.DataFrame(gen_data.detach().squeeze(-1).numpy())
        df_tot_fake = pd.concat([df_tot_fake, df_sub_fake], axis=0)

    # Post-processing
    df_post_processed = post_processing(df_tot_fake, **post_processing_config)
    
    # Hedging
    val_loss = hedging(df_post_processed)

    return val_loss



In [14]:
wandb.agent(sweep_id, function=objective, count=50)

wandb: Agent Starting Run: vd5q18ph with config:
wandb: 	D_hidden_dim: 32
wandb: 	D_num_layers: 2
wandb: 	E_hidden_dim: 16
wandb: 	E_num_layers: 3
wandb: 	learning_rate: 0.018296830688727863


Run vd5q18ph errored:
Traceback (most recent call last):
  File "/Users/sondrerogde/Dev/master-thesis/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/var/folders/3t/5vvh8l5x48s7tyvbx4b6v9xr0000gn/T/ipykernel_24212/2604151482.py", line 30, in objective
    train_pipeline = TrainingPipeline(model=model, training_config=training_config, exp_config=exp_config)
                                            ^^^^^
UnboundLocalError: cannot access local variable 'model' where it is not associated with a value

wandb: ERROR Run vd5q18ph errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/Users/sondrerogde/Dev/master-thesis/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/var/folders/3t/5vvh8l5x48s7tyvbx4b6v9xr0000gn/T/ipykernel_24212/2604151482.py", line 30, in objective
wandb: ERROR     train_pipeline = TrainingPipel

Run 9tetuont errored:
Traceback (most recent call last):
  File "/Users/sondrerogde/Dev/master-thesis/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/var/folders/3t/5vvh8l5x48s7tyvbx4b6v9xr0000gn/T/ipykernel_24212/2604151482.py", line 30, in objective
    train_pipeline = TrainingPipeline(model=model, training_config=training_config, exp_config=exp_config)
                                            ^^^^^
UnboundLocalError: cannot access local variable 'model' where it is not associated with a value

wandb: ERROR Run 9tetuont errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/Users/sondrerogde/Dev/master-thesis/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/var/folders/3t/5vvh8l5x48s7tyvbx4b6v9xr0000gn/T/ipykernel_24212/2604151482.py", line 30, in objective
wandb: ERROR     train_pipeline = TrainingPipel

Run bhkk0w6m errored:
Traceback (most recent call last):
  File "/Users/sondrerogde/Dev/master-thesis/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/var/folders/3t/5vvh8l5x48s7tyvbx4b6v9xr0000gn/T/ipykernel_24212/2604151482.py", line 30, in objective
    train_pipeline = TrainingPipeline(model=model, training_config=training_config, exp_config=exp_config)
                                            ^^^^^
UnboundLocalError: cannot access local variable 'model' where it is not associated with a value

wandb: ERROR Run bhkk0w6m errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/Users/sondrerogde/Dev/master-thesis/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/var/folders/3t/5vvh8l5x48s7tyvbx4b6v9xr0000gn/T/ipykernel_24212/2604151482.py", line 30, in objective
wandb: ERROR     train_pipeline = TrainingPipel